Logistic regression model using columns 'C1',
                 'banner_pos',
                 'site_category',
                 'app_category',
                 'device_type',
                 'device_conn_type',
                 'C15',
                 'C16',
                 'C18',
                 'C19',
                 'C21'.
                 
### Model Stats
- 01m55s to train 'small' for 2 parameters.
- 05m22s to train 'small' for 6 parameters.
- 07m21s to train 'small' for 10 parameters.
- {0.0001: -0.43356715906855536,
  0.001: -0.42629521272437554,
  0.01: -0.4242657276526611,
  0.1: -0.42427721774829286,
  1.0: -0.4245292557309675,
  10.0: -0.4246627635916426,
  100.0: -0.42472018597787536,
  1000.0: -0.42473047099609146,
  10000.0: -0.4247623788457422}
- {0.001: -0.42629521272437554,
  0.0021544346900318843: -0.4252131028945746,
  0.004641588833612777: -0.42457692978190453,
  0.01: -0.4242657276526611,
  0.021544346900318832: -0.4241657671706842,
  0.046415888336127774: -0.42418784783505004,
  0.1: -0.42427721774829286,
  0.21544346900318823: -0.4243908104081527,
  0.46415888336127775: -0.4244788065512767,
  1.0: -0.4245292557309675},
  Test score = -0.4251768174185952
- Submission trained on train_small:
    + Private 0.4168748
    + Public 0.4184437
- Train on 0.5 of the data:
    + Time: 10m10s.
    + Used around 20GB of memory.
    + Private 0.4172254
    + Public 0.4188042

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from tools.cv_tools import (
    fit_and_score,
    train_test_split, score_one_param, score_one_test_day,
    score_one_param, score_params, best_param
)

import time

## Use Pipeline
Since we got rid of LabelEncoders by upgrading sklearn to 0.20, we can implement the model as a Pipeline.

In [2]:
def eval_model_one(df, params):
    """
    df: pd.DataFrame. An output of pd.read_csv('train.csv') with its hour column formatted.
    """
    model_one_cols = ['C1',
                 'banner_pos',
                 'site_category',
                 'app_category',
                 'device_type',
                 'device_conn_type',
                 'C15',
                 'C16',
                 'C18',
                 'C19',
                 'C21']
    
    clicks = df.click
    # need the hour column for splitting
    df = df[model_one_cols + ['hour']]
    # Day 30 is for testing
    X_train, y_train, X_test, y_test = train_test_split(df, clicks, 30)
    test_day_ls = [25,26,27,28,29]
    
    
    oh_encoder = OneHotEncoder(handle_unknown='ignore')
    lg = LogisticRegression(solver='lbfgs')
    pipeline = Pipeline([('one_hot_encoding', oh_encoder),
                     ('logistic_regression', lg)])
    
    C_kwd = 'logistic_regression__C'
    params_dict_ls = [{C_kwd: p} for p in params]
    
    train_begin = time.time()
    scores = score_params(X_train, y_train, pipeline, params_dict_ls, test_day_ls)
    train_time = time.time() - train_begin
    print("Train time: ", train_time)
    best_C = best_param(scores, params_dict_ls)
    print("Best C: ", best_C)
    
    # Use the best parameter to evaluate the model on the test set.
    test_begin = time.time()
    test_score = fit_and_score(X_train, y_train, X_test, y_test, pipeline, best_C)
    test_time = time.time() - test_begin
    print("Test time: ", test_time)
    
    return params_dict_ls, scores, test_score

In [3]:
df = pd.read_csv('data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")
params = np.logspace(-3, 0, num=10)
params

array([0.001     , 0.00215443, 0.00464159, 0.01      , 0.02154435,
       0.04641589, 0.1       , 0.21544347, 0.46415888, 1.        ])

In [4]:
params_dict_ls, scores, test_score = eval_model_one(df, params)
best_C = best_param(scores, params_dict_ls)
best_C, dict(zip(params, scores)), test_score

Train time:  1.7080597877502441
Best C:  {'logistic_regression__C': 0.21544346900318823}
Test time:  0.03762531280517578


({'logistic_regression__C': 0.21544346900318823},
 {0.001: -0.42108051889201825,
  0.0021544346900318843: -0.4198515096584604,
  0.004641588833612777: -0.41779683762692804,
  0.01: -0.41495101836016646,
  0.021544346900318832: -0.4119152744321563,
  0.046415888336127774: -0.4095813893326916,
  0.1: -0.40828616903588627,
  0.21544346900318823: -0.40769989663579514,
  0.46415888336127775: -0.4079447501332506,
  1.0: -0.41041336356420555},
 -0.4571549241971712)

In [5]:
raise

RuntimeError: No active exception to reraise

## Develop cross validation
Test set = Day 30. Days 20-29 are partitioned into train and dev sets.

| Train | Dev |
|:-----:|:----:|
| 20-28 | 29 |
| 20-27 | 28 |
| ...   | ...|
| 20-24 | 25 |

In [ ]:
# helper functions

# this is specific to the model
def fit_transform_train(X_train):
    X_train = X_train.values
    # step 2: apply one-hot encoding
    # when transforming, an unknown categorical feature is mapped to a zero vector
    oh_encoder = OneHotEncoder(handle_unknown='ignore')
    X_train = oh_encoder.fit_transform(X_train)
    return X_train, oh_encoder

# this is specific to the model
def transform_dev(X_dev, oh_encoder):
    X_dev = oh_encoder.transform(X_dev)
    return X_dev

In [ ]:
def _fit_and_score(X_train, y_train, X_dev, y_dev, param):
    # the DataFrame needs the hour column for splitting. Drop the column right before training.
    X_train = X_train.drop('hour', axis=1)
    X_dev = X_dev.drop('hour', axis=1)
    
    X_train, oh_encoder = fit_transform_train(X_train)
    lg = LogisticRegression(C=param)
    lg.fit(X_train, y_train)
    X_dev = transform_dev(X_dev, oh_encoder)
    return neg_log_loss_score(lg, X_dev, y_dev)
    
def score_one_test_day(X_train, y_train, test_day, param):
    X_train, y_train, X_dev, y_dev = train_test_split(X_train, y_train, test_day)
    return _fit_and_score(X_train, y_train, X_dev, y_dev, param)

def score_one_param(X_train, y_train, param, test_day_ls):
    
    scores = []
    for test_day in test_day_ls:
        scores.append(score_one_test_day(X_train, y_train, test_day, param))

    assert len(scores) == len(test_day_ls)

    #print('Scores:', scores)
    mean_score = sum(scores)/len(scores)
    #print('Mean: ', mean_score)
    return mean_score
    
def score_params(X_train, y_train, param_ls, test_day_ls):
    """
    Returns a map: parameter -> mean score.
    """
    
    result = dict()
    for param in param_ls:
        result[param] = score_one_param(X_train, y_train, param, test_day_ls)
    return result

def best_param(score_dict):
    return max(score_dict.items(), key=operator.itemgetter(1))[0]

In [ ]:
def eval_model_one(df, params):
    """
    df: pd.DataFrame. An output of pd.read_csv('train.csv') with its hour column formatted.
    """
    model_one_cols = ['C1',
                 'banner_pos',
                 'site_category',
                 'app_category',
                 'device_type',
                 'device_conn_type',
                 'C15',
                 'C16',
                 'C18',
                 'C19',
                 'C21']
    
    clicks = df.click
    # need the hour column for splitting
    df = df[model_one_cols + ['hour']]
    # Day 30 is for testing
    X_train, y_train, X_test, y_test = train_test_split(df, clicks, 30)
    test_day_ls = [25,26,27,28,29]
    
    
    train_begin = time.time()
    scores = score_params(X_train, y_train, params, test_day_ls)
    train_time = time.time() - train_begin
    print("Train time: ", train_time)
    best_C = best_param(scores)
    print("Best C: ", best_C)
    
    # Use the best parameter to evaluate the model on the test set.
    test_begin = time.time()
    test_score = _fit_and_score(X_train, y_train, X_test, y_test, best_C)
    test_time = time.time() - test_begin
    print("Test time: ", test_time)
    
    return scores, test_score

In [ ]:
df = pd.read_csv('data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")

In [ ]:
#params = [0.001]
#K = 4
#params = np.logspace(-K, K, num=K*2+1)
params = np.logspace(-3, 0, num=10)
params

In [ ]:
%%time
scores, test_score = eval_model_one(df, params)

In [ ]:
best_C = best_param(scores)
best_C, scores, test_score

In [ ]:
raise

## Model 1 (baseline)

In [ ]:
# create a map: colName -> index
def build_colname_idx(df):
    return dict(zip(df.columns, range(len(df.columns))))

model1_cols = ['C1',
             'banner_pos',
             'site_category',
             'app_category',
             'device_type',
             'device_conn_type',
             'C15',
             'C16',
             'C18',
             'C19',
             'C21']

str_cols = ['site_category', 'app_category']

In [ ]:
df = pd.read_csv('data/train_tiny.csv')
df.hour = pd.to_datetime(df.hour, format="%y%m%d%H")

In [ ]:
def fit_encoders(df):
    site_category_encoder = LabelEncoder()
    site_category_encoder.fit(df.site_category)
    app_category_encoder = LabelEncoder()
    app_category_encoder.fit(df.app_category)
    return site_category_encoder, app_category_encoder

In [ ]:
# LabelEncoder doesn't handle unknown values, so fit them to the entire dataset.
site_category_encoder, app_category_encoder = fit_encoders(df)

In [ ]:
def train_dev_test_split(df):
    last_day = 30
    last_day_mask = df.hour.dt.day == last_day
    df_test = df[last_day_mask]
    df_test = df_test[model1_cols + ['click']]
    df_dev, df_test = train_test_split(df_test, test_size=0.5, random_state=23)
    df_train = df[~last_day_mask]
    df_train = df_train[model1_cols + ['click']]
    X_train = df_train.drop('click', axis=1)
    y_train = df_train.click
    X_dev = df_dev.drop('click', axis=1)
    y_dev = df_dev.click
    X_test = df_test.drop('click', axis=1)
    y_test = df_test.click
    return X_train, y_train, X_dev, y_dev, X_test, y_test

In [ ]:
# train/dev/test split
X_train, y_train, X_dev, y_dev, X_test, y_test = train_dev_test_split(df)

In [ ]:
def fit_transform_train(X_train, site_category_encoder, app_category_encoder):
    X_train.site_category = site_category_encoder.transform(X_train.site_category)
    X_train.app_category = app_category_encoder.transform(X_train.app_category)
    X_train = X_train.values
    # when transforming, an unknown categorical feature is mapped to a zero vector
    oh_encoder = OneHotEncoder(handle_unknown='ignore')
    X_train = oh_encoder.fit_transform(X_train)
    return X_train, oh_encoder

In [ ]:
train_colname_idx = build_colname_idx(X_train)
train_colname_idx

In [ ]:
# prepare the train set
X_train, oh_encoder = fit_transform_train(X_train, site_category_encoder, app_category_encoder)

In [ ]:
# tune: C. Maybe use class_weight. Try different solvers.
# Can be parallelized if multi-class.
lg = LogisticRegression()
lg.fit(X_train, y_train)

In [ ]:
def transform_dev(X_dev, site_category_encoder, app_category_encoder, oh_encoder):
    X_dev.site_category = site_category_encoder.transform(X_dev.site_category)
    X_dev.app_category = app_category_encoder.transform(X_dev.app_category)
    X_dev = oh_encoder.transform(X_dev)
    return X_dev

In [ ]:
# prepare the dev set
X_dev = transform_dev(X_dev, site_category_encoder, app_category_encoder, oh_encoder)

In [ ]:
lg.score(X_dev, y_dev)